In [1]:
import re

In [243]:
    def _get_regex():
        # dim = '(x?)(\s?)(\d+)([.|,]?)((\d+)?)(\s?)'
        # dim_2 = dim + '(\s?)(x)(\s?)' + dim
        # dim_3 = dim_2 + '(\s?)(x)(\s?)' + dim
        # ev_dim = '|'.join([dim, dim_2, dim_3])

        # definimos las magnitudes
        magnitud = '\d+[.|,]?\d*'

        # definimos la unidad
        longitud = 'mm|mtrs|m|cm|km'
        volumen = 'ltrs|l|m3|cc'
        masa = 'kg|grs|gr|g'
        bytes_ = 'b|mb|tb|gb'

        # unidad = '|'.join([longitud, volumen, masa, bytes_])

        # posible union "10cmX15cmx1m"
        union = 'x'

        non_alpha = re.compile(r'[^0-9a-zA-Záéíóú]+|\s+|\n+')
        
        all_metrics = '|'.join([longitud, volumen, masa, bytes_])
        nada_metric = re.compile('(^a-zA-Z.,)(({})+(?!{}){}?)+(?!a-zA-Z)'.format(magnitud, all_metrics, union))
 
        years = re.compile(r'''[0-9]{4}(?!a-zA-Z.,0-9)''')
    
        agregates = [(' ', non_alpha), (' año ', years), (' unknown_metric ', nada_metric)]
        # una forma mas linda
        # # unidades de metricas
        metric_tmpl = '(({})+({})[^0-9]+{}?)+'
        #
        unidades = [(bytes_, ' <BYTES> '), (volumen, ' <VOLUMEN> '), (masa, ' <MASS> '), (longitud, ' <LONGITUD> ')]
        metricas = []
        for unidad, tag in unidades:
            metrica = metric_tmpl.format(magnitud, unidad, union)
            metricas.append((tag, re.compile(metrica)))

        return agregates + metricas

In [244]:
regexesss = _get_regex()
def tokenize(title):
    metrics = regexesss
    for token, regex in metrics:
        print(title)
        title = re.sub(regex, token, title)
    return title


In [245]:
%%time 
tokenize('capot corolla 2002 2003 2004 2005 2006 2007 2008')

capot corolla 2002 2003 2004 2005 2006 2007 2008
capot corolla 2002 2003 2004 2005 2006 2007 2008
capot corolla  año   año   año   año   año   año   año 
capot corolla  año   año   año   año   año   año   año 
capot corolla  año   año   año   año   año   año   año 
capot corolla  año   año   año   año   año   año   año 
capot corolla  año   año   año   año   año   año   año 
CPU times: user 0 ns, sys: 0 ns, total: 0 ns
Wall time: 522 µs


'capot corolla  año   año   año   año   año   año   año '

In [246]:
%%time 
tokenize('Hp Eg0300fbdbr 597609-001 0b25642 300gb 2.5  10k 6g Dual Por')

Hp Eg0300fbdbr 597609-001 0b25642 300gb 2.5  10k 6g Dual Por
Hp Eg0300fbdbr 597609 001 0b25642 300gb 2 5 10k 6g Dual Por
Hp Eg año fbdbr  año 09 001 0b año 2 300gb 2 5 10k 6g Dual Por
Hp Eg año fbdbr  año 09 001 0b año 2 300gb 2 5 10k 6g Dual Por
Hp Eg año fbdbr  año 09 001  <BYTES> 2  <BYTES> 2 5 10k 6g Dual Por
Hp Eg año fbdbr  año 09 001  <BYTES> 2  <BYTES> 2 5 10k 6g Dual Por
Hp Eg año fbdbr  año 09 001  <BYTES> 2  <BYTES> 2 5 10k  <MASS> 
CPU times: user 4 ms, sys: 0 ns, total: 4 ms
Wall time: 940 µs


'Hp Eg año fbdbr  año 09 001  <BYTES> 2  <BYTES> 2 5 10k  <MASS> '